# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame

In [2]:
cities_df = pd.read_csv("..\WeatherPy\cities.csv",usecols=["City","Latitude","Longitude","Max Temp",\
                                              "Humidity","Cloudiness","Wind Speed","Country"]) 
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Palmerston,-12.49,130.98,64.40,88,0,2.24,AU
1,Kapaa,22.08,-159.32,82.40,74,20,11.41,US
2,Avarua,-21.21,-159.78,75.20,94,5,4.70,CK
3,Bambous Virieux,-20.34,57.76,66.99,77,75,18.34,MU
4,Busselton,-33.65,115.33,43.00,76,0,4.76,AU
...,...,...,...,...,...,...,...,...
565,Mandalgovi,45.76,106.27,71.53,50,97,5.59,MN
566,Santander Jimenez,24.22,-98.47,98.98,35,20,16.17,MX
567,Morant Bay,17.88,-76.41,80.60,74,75,19.46,JM
568,Falkensee,52.57,13.08,62.60,55,10,1.12,DE


### Humidity Heatmap

In [3]:
locations = cities_df[["Latitude","Longitude"]]
rating = cities_df["Humidity"].astype(float)

In [4]:
print(len(locations))
print(len(rating))

570
570


In [5]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2,center=(20,0))
# fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [6]:
ideal_df = cities_df[cities_df.Humidity <= 75]
ideal_df = ideal_df[ideal_df["Humidity"] >= 40]
ideal_df = ideal_df[ideal_df["Wind Speed"] <= 10]
ideal_df = ideal_df[ideal_df["Max Temp"] <= 75]
ideal_df = ideal_df[ideal_df["Max Temp"] >= 70]
ideal_df = ideal_df[ideal_df["Cloudiness"] <= 30]
ideal_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
111,Lata,40.16,-8.33,73.99,65,0,5.99,PT
116,Lagoa,39.05,-27.98,70.07,70,0,4.25,PT
270,Barbate,36.19,-5.92,71.01,54,0,4.50,ES
273,Praia Da Vitoria,38.73,-27.07,71.60,64,20,3.91,PT
307,Khash,28.22,61.22,70.56,44,0,1.86,IR
313,Akdepe,42.06,59.38,73.40,64,10,4.70,TM
351,Lompoc,34.64,-120.46,71.60,56,0,4.70,US
428,Nyanguge,-2.55,33.20,71.60,73,20,5.82,TZ
480,El Sauzal,31.90,-116.68,73.00,68,0,4.00,MX
555,Naqadeh,36.96,45.39,72.84,58,0,6.44,IR


### Hotel Map

In [7]:
ideal_df["Hotel Name"] = ""
ideal_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
111,Lata,40.16,-8.33,73.99,65,0,5.99,PT,
116,Lagoa,39.05,-27.98,70.07,70,0,4.25,PT,
270,Barbate,36.19,-5.92,71.01,54,0,4.50,ES,
273,Praia Da Vitoria,38.73,-27.07,71.60,64,20,3.91,PT,
307,Khash,28.22,61.22,70.56,44,0,1.86,IR,
313,Akdepe,42.06,59.38,73.40,64,10,4.70,TM,
351,Lompoc,34.64,-120.46,71.60,56,0,4.70,US,
428,Nyanguge,-2.55,33.20,71.60,73,20,5.82,TZ,
480,El Sauzal,31.90,-116.68,73.00,68,0,4.00,MX,
555,Naqadeh,36.96,45.39,72.84,58,0,6.44,IR,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
my_params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in ideal_df.iterrows():
    coordinates = f'{row["Latitude"]},{row["Longitude"]}'
    my_params["location"] = coordinates
    response = requests.get(base_url, params=my_params).json()
    results = response['results']
    ideal_df.loc[index, 'Hotel Name'] = results[0]['name']
    ideal_df.loc[index, 'address'] = results[0]['vicinity']

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
coords = ideal_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coords,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))